<a href="https://colab.research.google.com/github/Nazmul1932/SPL_3/blob/main/ML_part_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/MichalDanielDobrzanski/DeepLearningPython35.git

Cloning into 'DeepLearningPython35'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 59 (delta 0), reused 2 (delta 0), pack-reused 55
Unpacking objects: 100% (59/59), done.


In [2]:
cd '/content/DeepLearningPython35/'

/content/DeepLearningPython35


In [4]:
import mnist_loader

In [5]:
# %load network.py

"""
network.py
~~~~~~~~~~
IT WORKS

A module to implement the stochastic gradient descent learning
algorithm for a feedforward neural network.  Gradients are calculated
using backpropagation.  Note that I have focused on making the code
simple, easily readable, and easily modifiable.  It is not optimized,
and omits many desirable features.
"""

#### Libraries
# Standard library
import random

# Third-party libraries
import numpy as np

class Network(object):

    def __init__(self, sizes,act_fun_name):
        self.act_name = act_fun_name
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
          if self.act_name == "sigmoid":
            # print("Sig feedforward")
            a = sigmoid(np.dot(w, a)+b)
          elif self.act_name == "relu":
            # print("Relu feedforward")
            a = relu(np.dot(w, a)+b)
          else:
            # print("Tanh feedforward")
            a = tanh(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        training_data = list(training_data)
        n = len(training_data)

        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)

        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {} : {} / {}".format(j,self.evaluate(test_data),n_test))
            else:
                print("Epoch {} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            if self.act_name == "sigmoid":
              # print("Sig backProp")
              activation = sigmoid(z)
            elif self.act_name == "relu":
              # print("Relu backProp")
              activation = relu(z)
            else:
              # print("Tanh backProp")
              activation = tanh(z)
            activations.append(activation)
        # backward pass
        # delta = self.cost_derivative(activations[-1], y) * \
        #     sigmoid_prime(zs[-1])

        if self.act_name == "sigmoid":
          # print("Sig backProp 1")
          delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        elif self.act_name == "relu":
          # print("Relu backProp 1")
          delta = self.cost_derivative(activations[-1], y) * \
            relu_prime(zs[-1])
        else:
          # print("Tanh backProp 1")
          delta = self.cost_derivative(activations[-1], y) * \
            tanh_prime(zs[-1])


        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        for l in range(2, self.num_layers):
            z = zs[-l]
            if self.act_name == "sigmoid":
              # print("Sig backProp 2")
              sp = sigmoid_prime(z)
            elif self.act_name == "relu":
              # print("Relu backProp 2")
              sp = relu_prime(z)
            else:
              # print("Tanh backProp 2")
              sp = tanh_prime(z)


            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))
  
def relu(z):
	return max(0.0, z)
 
def relu_prime(z):
  return np.greater(z, 0).astype(int)

def tanh(z):
    return (np.exp(z) - np.exp(-z)) / (np.exp(z) + np.exp(-z))

def tanh_prime(z):
    return 1 - np.power(tanh(z),2)

In [7]:
parameter_list = [784 , 10]
for act_name in [ "sigmoid","tanh"]:
  for learning_rate in [2.0,0.8,4.5]:
    print("Function = ",act_name, " and LR = ", learning_rate)
    for layer in range(3):

      print("layer ", layer+1)
      training_data , validation_data , test_data = mnist_loader . load_data_wrapper ()
      parameter_list.insert(1,10)
      net = Network (parameter_list, act_name)
      net.SGD(training_data , 40, 10, learning_rate, test_data = test_data )
  
 

Function =  sigmoid  and LR =  2.0
layer  1
Epoch 0 : 8653 / 10000
Epoch 1 : 8891 / 10000
Epoch 2 : 8965 / 10000
Epoch 3 : 9047 / 10000
Epoch 4 : 9090 / 10000
Epoch 5 : 9048 / 10000
Epoch 6 : 9099 / 10000
Epoch 7 : 9087 / 10000
Epoch 8 : 9147 / 10000
Epoch 9 : 9128 / 10000
Epoch 10 : 9088 / 10000
Epoch 11 : 9079 / 10000
Epoch 12 : 9099 / 10000
Epoch 13 : 9075 / 10000
Epoch 14 : 9162 / 10000
Epoch 15 : 9123 / 10000
Epoch 16 : 9154 / 10000
Epoch 17 : 9154 / 10000
Epoch 18 : 9140 / 10000
Epoch 19 : 9147 / 10000
Epoch 20 : 9174 / 10000
Epoch 21 : 9110 / 10000
Epoch 22 : 9191 / 10000
Epoch 23 : 9222 / 10000
Epoch 24 : 9191 / 10000
Epoch 25 : 9185 / 10000
Epoch 26 : 9205 / 10000
Epoch 27 : 9191 / 10000
Epoch 28 : 9215 / 10000
Epoch 29 : 9230 / 10000
Epoch 30 : 9244 / 10000
Epoch 31 : 9233 / 10000
Epoch 32 : 9272 / 10000
Epoch 33 : 9245 / 10000
Epoch 34 : 9258 / 10000
Epoch 35 : 9262 / 10000
Epoch 36 : 9235 / 10000
Epoch 37 : 9234 / 10000
Epoch 38 : 9250 / 10000
Epoch 39 : 9240 / 10000
layer 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:186: RuntimeWarning: overflow encountered in exp


Epoch 15 : 5441 / 10000
Epoch 16 : 5300 / 10000
Epoch 17 : 5532 / 10000
Epoch 18 : 5766 / 10000
Epoch 19 : 5691 / 10000
Epoch 20 : 3433 / 10000
Epoch 21 : 5332 / 10000
Epoch 22 : 5092 / 10000
Epoch 23 : 5072 / 10000
Epoch 24 : 4192 / 10000
Epoch 25 : 5303 / 10000
Epoch 26 : 3337 / 10000
Epoch 27 : 4599 / 10000
Epoch 28 : 1842 / 10000
Epoch 29 : 1708 / 10000
Epoch 30 : 1686 / 10000
Epoch 31 : 1874 / 10000
Epoch 32 : 1838 / 10000
Epoch 33 : 1136 / 10000
Epoch 34 : 981 / 10000
Epoch 35 : 1011 / 10000
Epoch 36 : 1136 / 10000
Epoch 37 : 1033 / 10000
Epoch 38 : 1033 / 10000
Epoch 39 : 1010 / 10000
layer  2
Epoch 0 : 3625 / 10000
Epoch 1 : 4583 / 10000
Epoch 2 : 4311 / 10000
Epoch 3 : 4528 / 10000
Epoch 4 : 3845 / 10000
Epoch 5 : 2549 / 10000
Epoch 6 : 3922 / 10000
Epoch 7 : 5001 / 10000
Epoch 8 : 5105 / 10000
Epoch 9 : 3300 / 10000
Epoch 10 : 4335 / 10000
Epoch 11 : 3238 / 10000
Epoch 12 : 3628 / 10000
Epoch 13 : 4945 / 10000
Epoch 14 : 4376 / 10000
Epoch 15 : 2977 / 10000
Epoch 16 : 3866 / 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:199: RuntimeWarning: overflow encountered in exp
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:199: RuntimeWarning: invalid value encountered in true_divide


Epoch 0 : 980 / 10000
Epoch 1 : 980 / 10000
Epoch 2 : 980 / 10000
Epoch 3 : 980 / 10000
Epoch 4 : 980 / 10000
Epoch 5 : 980 / 10000
Epoch 6 : 980 / 10000
Epoch 7 : 980 / 10000
Epoch 8 : 980 / 10000
Epoch 9 : 980 / 10000
Epoch 10 : 980 / 10000
Epoch 11 : 980 / 10000
Epoch 12 : 980 / 10000
Epoch 13 : 980 / 10000
Epoch 14 : 980 / 10000
Epoch 15 : 980 / 10000
Epoch 16 : 980 / 10000
Epoch 17 : 980 / 10000
Epoch 18 : 980 / 10000
Epoch 19 : 980 / 10000
Epoch 20 : 980 / 10000
Epoch 21 : 980 / 10000
Epoch 22 : 980 / 10000
Epoch 23 : 980 / 10000
Epoch 24 : 980 / 10000
Epoch 25 : 980 / 10000
Epoch 26 : 980 / 10000
Epoch 27 : 980 / 10000
Epoch 28 : 980 / 10000
Epoch 29 : 980 / 10000
Epoch 30 : 980 / 10000
Epoch 31 : 980 / 10000
Epoch 32 : 980 / 10000
Epoch 33 : 980 / 10000
Epoch 34 : 980 / 10000
Epoch 35 : 980 / 10000
Epoch 36 : 980 / 10000
Epoch 37 : 980 / 10000
Epoch 38 : 980 / 10000
Epoch 39 : 980 / 10000
layer  2
Epoch 0 : 1028 / 10000
Epoch 1 : 1028 / 10000
Epoch 2 : 1028 / 10000
Epoch 3 : 10